In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from finance_byu.summarize import summary
import pyarrow
# !pip uninstall pandas
# !pip install pandas==2.2.3
print(pd.__version__)

load_dotenv()  # reads .env file
SAVE_PATH = os.getenv("SAVE_PATH")

2.2.3


In [2]:
df = pd.read_feather(SAVE_PATH)

In [4]:
df['logret'] = np.log(1 + df['ret'])
df['mom'] = df.groupby('permno')['logret'].rolling(6,6).sum().reset_index(drop=True)
df['mom'] = df.groupby('permno')['mom'].shift(2)
df['me'] = df['shr'] * df['prc']

In [5]:
df['prclag'] = df.groupby('permno')['prc'].shift()
df['melag'] = df.groupby('permno')['me'].shift()
df['cashtoasset'] = df["cash"] / df["assets"]
df['currenttoasset'] = df["current"] / df["assets"]
df.drop(columns=["cash", "current", "assets"])

df = df.query("mom == mom and prclag >= 5").reset_index(drop=True)

KeyboardInterrupt: 

In [ ]:
df["mom_first"] = df.groupby('caldt')['mom'].transform(pd.qcut,5,labels=[str(k) for k in range(1, 6)])
df["cashtoasset_first"] = df.groupby('caldt')['cashtoasset'].transform(pd.qcut,5,labels=["a", "b", "c", "d", "e"])
df["currenttoasset_first"] = df.groupby('caldt')['currenttoasset'].transform(pd.qcut,5,labels=["a", "b", "c", "d", "e"])
df["cashtoasset_second"] = df.groupby(['caldt', 'mom_first'])['cashtoasset'].transform(pd.qcut,5,labels=["a", "b", "c", "d", "e"])
df["currenttoasset_second"] = df.groupby(['caldt', 'mom_first'])['currenttoasset'].transform(pd.qcut,5,labels=["a", "b", "c", "d", "e"])

In [ ]:
# Equal weighted, cash to asset ratio
port = df.groupby(['caldt','mom_first','cashtoasset_second'])['ret'].mean()
port = port.unstack(level=['mom_first','cashtoasset_second'])
print('Summary statistics from month to month, mom-cashasset:')
summary(port).round(4)

NameError: name 'df' is not defined